# Config & Imports

In [1]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
from tensorflow import keras
import keras.layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.callbacks import EarlyStopping

from keras.regularizers import L1L2
from keras.regularizers import L2

import utilities_LR

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

import sys

In [21]:
config = {
    'data': {
        'n_datasets': 10_000, # the number of datasets
        
        'n_samples': 5_000, # the number of samples per dataset
        
        'n_features': 15, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        #'n_informative': random.randint(2, 10),
        'n_informative': 'random',
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        # or random
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 1,
        # The number of clusters per class.
        
        'class_sep': 3.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'noise': 0,
        # flip_y (fraction of samples whose class is assigned randomly)
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 42,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': -1, # currently not used
                'val_size': 0.15,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': keras.losses.BinaryCrossentropy(from_logits=False), #tf.keras.losses.get(config['lambda_net']['loss_lambda']), # 'mae'
            'metrics': ['mae', keras.losses.BinaryCrossentropy(from_logits=False)]
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 1000,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.1,
                'val_size': 0.05,
                'random_state': None,
                'shuffle': False,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 100,
        'use_gpu': False,
        'gpu_numbers': '5',
        'RANDOM_SEED': 1,   
    }
}

In [22]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

In [23]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [24]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  0
Num XLA-GPUs Available:  0


# Functions

In [25]:
def load_lambda():
    directory = utilities_LR.lambda_path_LR(config)
    
    return np.load(directory + '/lambda_weights_list.npy', allow_pickle=True)

In [26]:
def get_X_data():
    directory = utilities_LR.data_path_LR(config)
    return np.load(directory + '/X_datasets_list_dataForLambda.npy', allow_pickle=True)

In [27]:
def get_y_pred_data():
    directory = utilities_LR.lambda_path_LR(config)
    
    #return np.load(directory + '/lambda_preds_list.npy', allow_pickle=True)
    return np.load(directory + '/lambda_preds_list_with0.2_noise.npy', allow_pickle=True)

# Model Training

In [28]:
X = load_lambda()

In [29]:
y_predictions = get_y_pred_data()

In [30]:
valid_feature_data = get_X_data()

In [31]:
y_predictions.shape

(10000, 5000)

In [32]:
valid_feature_data.shape

(10000, 5000, 15)

In [33]:
#valid_feature_data = np.hstack(valid_feature_data)
indexcol = np.array([np.arange(config['data']['n_datasets'], dtype=int)], ndmin=2).reshape(config['data']['n_datasets'], 1)
# print(indexcol.shape)
y_predictions = np.concatenate([indexcol, y_predictions], axis=1)

In [47]:
import pandas as pd

In [50]:
y_predictions = [[0 if y<= 0.5 else 1 for y in x] for x in y_predictions]

In [56]:
tmp = pd.DataFrame(y_predictions).transpose()

In [58]:
tmp

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,0,0,1,1,0,1,...,0,0,0,0,0,1,1,1,0,1
2,1,1,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,1,0,0,1
3,0,1,0,0,0,0,0,1,1,0,...,1,0,1,0,1,1,1,1,1,0
4,1,0,1,0,1,1,0,1,1,0,...,1,1,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0,0,0,1,1,1,0,0,1,1,...,0,0,1,1,0,1,0,1,0,0
4997,0,1,0,0,0,0,1,0,0,0,...,0,1,0,1,0,0,1,0,0,1
4998,0,1,0,0,0,0,0,1,1,1,...,1,1,1,0,0,0,0,1,1,0
4999,1,0,0,0,0,1,1,1,0,0,...,1,0,0,1,1,0,1,0,1,1


In [34]:
y_predictions.shape

(10000, 5001)

In [35]:
valid_feature_data = tf.convert_to_tensor(valid_feature_data, dtype=tf.float32)

# Custom Loss

In [36]:
@tf.function
def custom_loss(y_predictions_index, y_coef_pred):
    
    index = y_predictions_index[:, 0]
    y_true = y_predictions_index[:, 1:]
    
    index = tf.cast(index, tf.int32)
    
    X_feature_data_samples = tf.gather(valid_feature_data, index)
    
    #y_pred = tf.math.sigmoid(tf.linalg.matvec(valid_feature_data_sample, y_coef_pred))
    
    #tf.print("weights:", y_coef_pred[:, 1:], output_stream=sys.stdout)
    #tf.print("bias:", y_coef_pred[:, 0], output_stream=sys.stdout)
    #
    y_pred = tf.transpose(tf.math.add(tf.transpose(tf.linalg.matvec(X_feature_data_samples, y_coef_pred[:, 1:])), y_coef_pred[:, 0]))
    
    #tf.print("bias:", y_pred, output_stream=sys.stdout)

    metric = tf.keras.losses.BinaryCrossentropy(
                                from_logits=True,
                                label_smoothing=0.0,
                                axis=-1,
                                reduction='auto',
                                name='binary_crossentropy')
    loss = metric(y_true, y_pred)
    return loss

In [37]:
#@tf.function
#def custom_loss(y_predictions_index, y_coef_pred):
#    
#    index = y_predictions_index[:, 0]
#    y_true = y_predictions_index[:, 1:]
#    
#    index = tf.cast(index, tf.int32)
#    
#    X_feature_data_samples = tf.gather(valid_feature_data, index)
#    
#    #y_pred = tf.math.sigmoid(tf.linalg.matvec(valid_feature_data_sample, y_coef_pred))
#    y_pred = tf.linalg.matvec(X_feature_data_samples, y_coef_pred)
#
#    metric = tf.keras.losses.BinaryCrossentropy(
#                                from_logits=True,
#                                label_smoothing=0.0,
#                                axis=-1,
#                                reduction='auto',
#                                name='binary_crossentropy')
#    loss = metric(y_true, y_pred)
#    return loss

In [42]:
X_train.shape

(9000, 2237)

In [44]:
# Data Prep

early_stopping = EarlyStopping(monitor='val_loss',
                                min_delta=0.001,
                                patience=25,
                                verbose=0,
                                mode='auto',
                                baseline=None,
                                restore_best_weights=True)


#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                              patience=5, min_lr=0.0001)
#

#X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
#                                                                      y)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y_predictions, 
                                                    test_size=config['inets']['data_prep']['train_test_val_split']['test_size'], 
                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])




# Model Def
model = Sequential()
# model.add(BatchNormalization())
regval = 0.001
model.add(Dense(2056, activation='swish', kernel_regularizer=L2(regval), bias_regularizer= L2(regval), input_dim=X_train.shape[1]))
#model.add(keras.layers.Dropout(0.1))
model.add(Dense(1028, activation='swish', kernel_regularizer=L2(regval), bias_regularizer= L2(regval)))
#model.add(keras.layers.Dropout(0.2))
model.add(Dense(1028, activation='swish', kernel_regularizer=L2(regval), bias_regularizer= L2(regval)))
#model.add(keras.layers.Dropout(0.2))
model.add(Dense(1028, activation='swish', kernel_regularizer=L2(regval), bias_regularizer=  L2(regval)))
#model.add(keras.layers.Dropout(0.1))

#model.add(Dense( 512, activation='swish'))
#model.add(keras.layers.Dropout(0.3))
#model.add(Dense(2500, activation='relu'))
#model.add(Dense(1500, activation='linear'))
#model.add(Dense(500, activation='relu'))
model.add(Dense(config['data']['n_features'] + 1, activation='linear'))


opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999, # 0.999
    epsilon=1e-08, # 1.0, 0.1, 0.01
    amsgrad=False,
    name='Adam',)

#opt = tf.keras.optimizers.SGD(
#    learning_rate=0.1, # 0.001
#    momentum=0.0,
#    nesterov=False,
#    name='SGD'
#)

model.compile(optimizer=opt, loss=custom_loss, metrics=[])
#model.build(input_shape=X_train.shape)
model.summary()

# Model fit
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=config['inets']['model_fit']['batch_size'],
                    epochs=config['inets']['model_fit']['epochs'],
                    verbose=config['inets']['model_fit']['verbose'],
                    callbacks=[early_stopping],
                    validation_split=config['inets']['data_prep']['train_test_val_split']['val_size'],
                    shuffle=config['inets']['model_fit']['shuffle'],
                    class_weight=config['inets']['model_fit']['class_weight'],
                    sample_weight=config['inets']['model_fit']['sample_weight'],
                    initial_epoch=config['inets']['model_fit']['initial_epoch'],
                    steps_per_epoch=config['inets']['model_fit']['steps_per_epoch'],
                    validation_steps=config['inets']['model_fit']['validation_steps'],
                    validation_batch_size=config['inets']['model_fit']['validation_batch_size'],
                    validation_freq=config['inets']['model_fit']['validation_freq'],
                    workers=config['computation']['n_jobs'],
                    #use_multiprocessing=True
                   )
print(history.history['val_loss'])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 2056)              4601328   
                                                                 
 dense_21 (Dense)            (None, 1028)              2114596   
                                                                 
 dense_22 (Dense)            (None, 1028)              1057812   
                                                                 
 dense_23 (Dense)            (None, 1028)              1057812   
                                                                 
 dense_24 (Dense)            (None, 16)                16464     
                                                                 
Total params: 8,848,012
Trainable params: 8,848,012
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
268/268 [======================

KeyboardInterrupt: 

In [ ]:
## Data Prep
#
#early_stopping = EarlyStopping(monitor='val_loss',
#                                min_delta=0.001,
#                                patience=25,
#                                verbose=0,
#                                mode='auto',
#                                baseline=None,
#                                restore_best_weights=True)
#
##X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
##                                                                      y)
#
#X_train, X_val, y_train, y_val = train_test_split(X, 
#                                                    y_predictions, 
#                                                    test_size=config['inets']['data_prep']['train_test_val_split']['val_size'], 
#                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
#                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
#                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
#
#
## Model Def
#model = Sequential()
##model.add(BatchNormalization())
#model.add(Dense(1028, activation='swish'))
#model.add(keras.layers.Dropout(0.4))
#model.add(Dense(1028, activation='swish'))
#model.add(keras.layers.Dropout(0.4))
#model.add(Dense(512, activation='swish'))
#model.add(keras.layers.Dropout(0.4))
#
##model.add(Dense( 512, activation='swish'))
##model.add(keras.layers.Dropout(0.3))
##model.add(Dense(2500, activation='relu'))
##model.add(Dense(1500, activation='linear'))
##model.add(Dense(500, activation='relu'))
#model.add(Dense(config['data']['n_features'], activation='linear'))
#
#
##opt = tf.keras.optimizers.Adam(
##    learning_rate=0.0001,
##    beta_1=0.9,
##    beta_2=0.999,
##    epsilon=1e-07,
##    amsgrad=False,
##    name='Adam',)
#
#opt = tf.keras.optimizers.SGD(
#    learning_rate=0.001,
#    momentum=0.0,
#    nesterov=False,
#    name='SGD'
#)
#
#model.compile(optimizer=opt, loss=custom_loss, metrics=[])
#model.build(input_shape=X_train.shape)
#model.summary()
#
## Model fit
#history = model.fit(x=X_train,
#                    y=y_train,
#                    batch_size=config['inets']['model_fit']['batch_size'],
#                    epochs=config['inets']['model_fit']['epochs'],
#                    verbose=config['inets']['model_fit']['verbose'],
#                    #callbacks=[early_stopping],
#                    validation_data=(X_val, y_val),
#                    shuffle=config['inets']['model_fit']['shuffle'],
#                    class_weight=config['inets']['model_fit']['class_weight'],
#                    sample_weight=config['inets']['model_fit']['sample_weight'],
#                    initial_epoch=config['inets']['model_fit']['initial_epoch'],
#                    steps_per_epoch=config['inets']['model_fit']['steps_per_epoch'],
#                    validation_steps=config['inets']['model_fit']['validation_steps'],
#                    validation_batch_size=config['inets']['model_fit']['validation_batch_size'],
#                    validation_freq=config['inets']['model_fit']['validation_freq'],
#                    workers=config['computation']['n_jobs'],
#                    #use_multiprocessing=True
#                   )
#print(history.history['val_loss'])
#

In [ ]:
def save_model(model):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    model.save(path + '/modelKeras')

In [ ]:
save_model(model)

In [ ]:
def save_testData(X_test, y_test):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    np.save(path + '/X_test', X_test)
    
    np.save(path + '/y_test', y_test)

In [ ]:
save_testData(X_test, y_test)